In [17]:
#!pip install pynmea2
# !pip install geopandas
# !brew install proj
# !pip install cartopy
# !pip install Cython
# !pip install geoplot
'''
import glob
import itertools
import json
import os
import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
from geopandas import GeoDataFrame
import geoplot as gplt
from IPython.display import Video
from matplotlib import animation
import plotly.express as px
import pynmea2
import requests
from shapely.geometry import Point, shape
import shapely.wkt
'''
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

import seaborn as sns

%matplotlib inline

DATA_PATH = '../input/google-smartphone-decimeter-challenge/'

In [4]:
sub = pd.read_csv(DATA_PATH + 'sample_submission.csv')
sub.head()

,phone,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.904611,-86.481078
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.904611,-86.481078
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.904611,-86.481078
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.904611,-86.481078
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.904611,-86.481078


In [5]:
!ls ../input/google-smartphone-decimeter-challenge/train

2020-05-14-US-MTV-1 2020-06-05-US-MTV-2 2020-09-04-US-SF-2  2021-04-26-US-SVL-1
2020-05-14-US-MTV-2 2020-06-11-US-MTV-1 2021-01-04-US-RWC-1 2021-04-28-US-MTV-1
2020-05-21-US-MTV-1 2020-07-08-US-MTV-1 2021-01-04-US-RWC-2 2021-04-28-US-SJC-1
2020-05-21-US-MTV-2 2020-07-17-US-MTV-1 2021-01-05-US-SVL-1 2021-04-29-US-MTV-1
2020-05-29-US-MTV-1 2020-07-17-US-MTV-2 2021-01-05-US-SVL-2 2021-04-29-US-SJC-2
2020-05-29-US-MTV-2 2020-08-03-US-MTV-1 2021-03-10-US-SVL-1
2020-06-04-US-MTV-1 2020-08-06-US-MTV-2 2021-04-15-US-MTV-1
2020-06-05-US-MTV-1 2020-09-04-US-SF-1  2021-04-22-US-SJC-1


In [6]:
df_sample_trail = pd.read_csv(DATA_PATH + "train/2020-05-14-US-MTV-1/Pixel4/Pixel4_derived.csv")
df_sample_trail_gt = pd.read_csv(DATA_PATH + "train/2020-05-14-US-MTV-1/Pixel4/ground_truth.csv")

In [7]:
df_sample_trail.head()

,collectionName,phoneName,millisSinceGpsEpoch,constellationType,svid,signalType,receivedSvTimeInGpsNanos,xSatPosM,ySatPosM,zSatPosM,xSatVelMps,ySatVelMps,zSatVelMps,satClkBiasM,satClkDriftMps,rawPrM,rawPrUncM,isrbM,ionoDelayM,tropoDelayM
0,2020-05-14-US-MTV-1,Pixel4,1273529464442,3,24,GLO_G1,1273529463363061857,-2.539901e+07,-6.925122e+05,-2.280430e+06,-325.826,156.040,3559.757,-468.084,0.001,2.379498e+07,11.992,1134.758,10.866,16.647
1,2020-05-14-US-MTV-1,Pixel4,1273529464442,6,13,GAL_E1,1273529463363970742,-5.199894e+06,-1.741927e+07,2.336128e+07,2239.305,700.815,1022.014,120171.076,0.000,2.352251e+07,1.799,-222.675,3.946,2.717
2,2020-05-14-US-MTV-1,Pixel4,1273529464442,1,5,GPS_L1,1273529463365539137,-2.179863e+06,-2.615488e+07,-3.437694e+06,325.541,-419.725,3129.012,-3793.067,-0.001,2.305231e+07,4.197,0.000,7.554,5.704
3,2020-05-14-US-MTV-1,Pixel4,1273529464442,6,15,GAL_E1,1273529463352380595,1.635469e+07,-1.047833e+07,2.234460e+07,2172.409,561.971,-1326.001,259937.286,-0.000,2.699715e+07,8.094,-222.675,7.968,9.151
4,2020-05-14-US-MTV-1,Pixel4,1273529464442,6,21,GAL_E1,1273529463364075083,-1.484042e+07,-2.110960e+07,1.449632e+07,-426.094,-1395.675,-2467.793,-179555.991,-0.001,2.349123e+07,2.698,-222.675,3.834,2.544


In [9]:
df_sample_trail_gt.head()

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,timeSinceFirstFixSeconds,hDop,vDop,speedMps,courseDegree
0,2020-05-14-US-MTV-1,Pixel4,1273529463442,37.423576,-122.094132,33.21,551.44,1.1,0.0,0.0,12.7
1,2020-05-14-US-MTV-1,Pixel4,1273529464442,37.423576,-122.094132,33.21,552.44,1.1,0.0,0.0,12.7
2,2020-05-14-US-MTV-1,Pixel4,1273529465442,37.423576,-122.094132,33.21,553.44,1.1,0.0,0.0,12.7
3,2020-05-14-US-MTV-1,Pixel4,1273529466442,37.423576,-122.094132,33.20,554.44,1.1,0.0,0.0,12.7
4,2020-05-14-US-MTV-1,Pixel4,1273529467442,37.423576,-122.094132,33.20,555.44,1.1,0.0,0.0,12.7
